# Introduction to GCP and Semantic Kernel

### Google Cloud Platform (GCP)
* Google Cloud Platform (GCP) is a suite of cloud computing services offered by Google, providing a range of infrastructure, platform, and serverless computing environments.
* It includes services like Google Cloud Storage (used in this notebook for file management), Compute Engine, BigQuery, and AI/ML tools. GCP enables scalable, secure, and efficient data storage, processing, and analysis, making it ideal for building and deploying applications, managing data, and integrating AI capabilities.

### Semantic Kernel
* Semantic Kernel is an open-source SDK developed by Microsoft that simplifies the integration of AI models and services into applications. * It acts as an orchestration layer, allowing developers to combine AI capabilities (like large language models from OpenAI) with traditional programming logic. It supports plugins, agents, and workflows, enabling the creation of intelligent applications that can process natural language, automate tasks, and integrate with external services like GCP.

### Semantic Kernel with GCP
* This notebook demonstrates the integration of Semantic Kernel with Google Cloud Platform (GCP) to create a multi-agent system for file management, analysis, and reporting.
* Below are the code cells and their outputs, as provided in the original notebook.

## Cell 1: Install Required Libraries

In [ ]:
!pip install semantic-kernel openai google-cloud-storage python-dotenv

## Cell 2: Import Libraries

In [2]:
import os
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from google.cloud import storage
import openai
from pathlib import Path

print("✅ All libraries imported")

✅ All libraries imported


## Cell 3: Set Up Environment and Initialize Semantic Kernel

In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "Your_Jason-File-Path"
os.environ["OPENAI_API_KEY"] = "Your_API_KEY"

# Create Semantic Kernel
kernel = sk.Kernel()
kernel.add_service(OpenAIChatCompletion(
    ai_model_id="gpt-4o",
    api_key=os.getenv("OPENAI_API_KEY"),
    service_id="gpt"
))

print("✅ Kernel ready")

✅ Kernel ready


## Cell 4: Define Enhanced GCP Agent

In [4]:
class EnhancedGCPAgent:
    """Enhanced GCP Agent with better file handling"""

    def __init__(self):
        self.client = storage.Client(project="Gen-ai")
        self.bucket_name = "bucket_demo8"
        self.base_dir = Path(r"Your_File_Upload_PAth")

    @kernel_function(description="List files in bucket and local directory")
    def list_files(self) -> str:
        # List files in bucket
        bucket = self.client.bucket(self.bucket_name)
        bucket_files = [blob.name for blob in bucket.list_blobs()]

        # List files in local directory
        local_files = []
        if self.base_dir.exists():
            local_files = [f.name for f in self.base_dir.iterdir() if f.is_file()]

        return f"Bucket files: {bucket_files}\nLocal files: {local_files}"

    @kernel_function(description="Upload file to bucket with smart filename matching")
    def upload_file(self, filename: str) -> str:
        """Upload file with smart filename matching"""
        try:
            # Try multiple filename variations
            possible_names = [
                filename,
                filename.replace('_', ' '),
                filename.replace(' ', '_'),
                filename.replace('eer2', 'eer_2'),
                filename.replace('eer_2', 'eer2')
            ]

            file_path = None
            actual_filename = None

            # Find the actual file
            for name in possible_names:
                test_path = self.base_dir / name
                if test_path.exists():
                    file_path = test_path
                    actual_filename = name
                    break

            if file_path is None:
                # List available files for user
                if self.base_dir.exists():
                    available_files = [f.name for f in self.base_dir.iterdir() if f.is_file()]
                    return f"❌ File '{filename}' not found. Available files: {available_files}"
                else:
                    return f"❌ Directory '{self.base_dir}' does not exist"

            # Upload the file
            bucket = self.client.bucket(self.bucket_name)
            blob = bucket.blob(filename)  # Use requested filename for blob
            blob.upload_from_filename(str(file_path))

            return f"✅ Uploaded '{actual_filename}' as '{filename}' to {self.bucket_name}"

        except Exception as e:
            return f"❌ Upload error: {str(e)}"

    @kernel_function(description="Get detailed file info")
    def get_file_info(self, filename: str) -> str:
        bucket = self.client.bucket(self.bucket_name)
        blob = bucket.blob(filename)
        if blob.exists():
            blob.reload()  # Get latest metadata
            return f"File: {filename}\nSize: {blob.size} bytes\nCreated: {blob.time_created}\nType: {blob.content_type}"
        return f"File {filename} not found in bucket"

    @kernel_function(description="Check local files in directory")
    def check_local_files(self) -> str:
        """Check what files are available locally"""
        if not self.base_dir.exists():
            return f"❌ Directory {self.base_dir} does not exist"

        files = []
        for file_path in self.base_dir.iterdir():
            if file_path.is_file():
                size = file_path.stat().st_size
                files.append(f"{file_path.name} ({size} bytes)")

        if files:
            return f"Local files in {self.base_dir}:\n" + "\n".join(files)
        else:
            return f"No files found in {self.base_dir}"

# Add Enhanced GCP Agent to kernel
gcp_agent = EnhancedGCPAgent()
kernel.add_plugin(gcp_agent, plugin_name="gcp")
print("✅ Enhanced GCP Agent added to Semantic Kernel")

✅ Enhanced GCP Agent added to Semantic Kernel


## Cell 5: Define Analysis Agent

In [5]:
class AnalysisAgent:
    """Analysis Agent using OpenAI"""

    def __init__(self):
        self.client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    def analyze_files(self, file_data):
        """Analyze files"""
        prompt = f"""
        You are a Data Analysis Agent. Analyze these files:

        {file_data}

        Provide:
        1. File types found
        2. Recommendations
        3. Next steps

        Keep it short.
        """

        response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=300
        )
        return response.choices[0].message.content

analysis_agent = AnalysisAgent()
print("✅ Analysis Agent created")

✅ Analysis Agent created


## Cell 6: Define Report Agent

In [6]:
class ReportAgent:
    """Report Agent using OpenAI"""

    def __init__(self):
        self.client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    def create_report(self, data):
        """Create report"""
        prompt = f"""
        You are a Report Agent. Create a brief summary:

        {data}

        Provide:
        1. Summary
        2. Key findings
        3. Recommended actions

        Keep it concise.
        """

        response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=300
        )
        return response.choices[0].message.content

report_agent = ReportAgent()
print("✅ Report Agent created")

✅ Report Agent created


## Cell 7: Define Multi-Agent Coordinator

In [7]:
class MultiAgentCoordinator:
    """Coordinates all agents using Semantic Kernel"""

    def __init__(self, kernel):
        self.kernel = kernel

    async def run_workflow(self):
        """Run multi-agent workflow"""
        print("🚀 Multi-Agent Workflow Starting...")

        # Step 1: GCP Agent gets files (REAL Semantic Kernel)
        print("\n📋 GCP Agent: Getting files...")
        files = await self.kernel.invoke(self.kernel.plugins["gcp"]["list_files"])
        print(f"GCP Result: {files}")

        # Step 2: Analysis Agent analyzes
        print("\n🧠 Analysis Agent: Analyzing...")
        analysis = analysis_agent.analyze_files(str(files))
        print(f"Analysis: {analysis[:100]}...")

        # Step 3: Report Agent creates report
        print("\n📊 Report Agent: Creating report...")
        report = report_agent.create_report(f"Files: {files}\nAnalysis: {analysis}")
        print(f"Report: {report[:100]}...")

        return {
            "files": str(files),
            "analysis": analysis,
            "report": report
        }

    async def upload_and_analyze(self, filename):
        """Upload and analyze using Semantic Kernel"""
        print(f"📤 Upload and Analyze: {filename}")

        # Check local files first
        local_check = await self.kernel.invoke(self.kernel.plugins["gcp"]["check_local_files"])
        print(f"Local files check: {local_check}")

        # Upload using REAL Semantic Kernel
        upload_result = await self.kernel.invoke(
            self.kernel.plugins["gcp"]["upload_file"],
            filename=filename
        )
        print(f"Upload: {upload_result}")

        # Get files using REAL Semantic Kernel
        files = await self.kernel.invoke(self.kernel.plugins["gcp"]["list_files"])

        # Analyze with AI agent
        analysis = analysis_agent.analyze_files(str(files))

        return {"upload": str(upload_result), "analysis": analysis}

# Create coordinator
coordinator = MultiAgentCoordinator(kernel)
print("✅ Multi-Agent Coordinator ready")

✅ Multi-Agent Coordinator ready


## Cell 8: Define Quick Commands

In [8]:
async def quick_list():
    """Quick file list using REAL Semantic Kernel"""
    files = await kernel.invoke(kernel.plugins["gcp"]["list_files"])
    print(files)
    return files

async def quick_check_local():
    """Check local files using REAL Semantic Kernel"""
    files = await kernel.invoke(kernel.plugins["gcp"]["check_local_files"])
    print(files)
    return files

async def quick_upload(filename):
    """Quick upload using REAL Semantic Kernel"""
    result = await kernel.invoke(kernel.plugins["gcp"]["upload_file"], filename=filename)
    print(result)
    return result

async def quick_analyze():
    """Quick analysis using multi-agents"""
    files = await kernel.invoke(kernel.plugins["gcp"]["list_files"])
    analysis = analysis_agent.analyze_files(str(files))
    print(analysis)
    return analysis

async def quick_workflow():
    """Quick full workflow"""
    return await coordinator.run_workflow()

async def quick_upload_and_analyze(filename):
    """Quick upload and analyze"""
    return await coordinator.upload_and_analyze(filename)

print("✅ Enhanced quick commands ready:")
print("- await quick_check_local()          # Check local files")
print("- await quick_list()                 # List bucket & local files")
print("- await quick_upload('filename')     # Smart upload")
print("- await quick_analyze()              # Multi-agent analysis")
print("- await quick_workflow()             # Full workflow")
print("- await quick_upload_and_analyze('filename')  # Upload + analyze")

✅ Enhanced quick commands ready:
- await quick_check_local()          # Check local files
- await quick_list()                 # List bucket & local files
- await quick_upload('filename')     # Smart upload
- await quick_analyze()              # Multi-agent analysis
- await quick_workflow()             # Full workflow
- await quick_upload_and_analyze('filename')  # Upload + analyze


## Cell 9: Test Enhanced System

In [10]:
print("\n=== Testing Enhanced System ===")

async def test_enhanced_system():
    """Test the enhanced system"""

    print("🔧 1. Checking local files...")
    local_files = await quick_check_local()

    print("\n📋 2. Listing bucket and local files...")
    all_files = await quick_list()

    print("\n📤 3. Testing upload with temp_eer_2.pdf...")
    upload_result = await quick_upload("temp_eer_2.pdf")

    print("\n🧠 4. Testing analysis...")
    analysis = await quick_analyze()

    print("\n🚀 5. Testing full workflow...")
    workflow = await quick_workflow()

    print("\n🎉 ALL TESTS COMPLETE!")
    print("\n🚀 Your Multi-Agent System is Ready! Use the commands above to interact with it.")

await test_enhanced_system()


=== Testing Enhanced System ===
🔧 1. Checking local files...
Local files in /content:
temp_eer_2.pdf (1229101 bytes)
gen-ai-462005-7ec6dfae325a.json (2357 bytes)

📋 2. Listing bucket and local files...
Bucket files: ['main.py']
Local files: ['temp_eer_2.pdf', 'gen-ai-462005-7ec6dfae325a.json']

📤 3. Testing upload with temp_eer_2.pdf...
✅ Uploaded 'temp_eer_2.pdf' as 'temp_eer_2.pdf' to bucket_demo8

🧠 4. Testing analysis...
1. **File Types Found:**
   - Python script: `main.py`
   - PDF document: `temp_eer_2.pdf`
   - JSON file: `gen-ai-462005-7ec6dfae325a.json`

2. **Recommendations:**
   - **For `main.py`:** Ensure the script is properly version-controlled and has clear comments/documentation for ease of maintenance.
   - **For `temp_eer_2.pdf`:** If this is a temporary or generated file, consider organizing or archiving it if it’s not frequently used, to reduce clutter.
   - **For `gen-ai-462005-7ec6dfae325a.json`:** Secure sensitive data within the JSON, especially if it contains